In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import json

In [2]:
word_dict = json.load(open("result.json"))

alphabet = 'A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z'
alphabet = alphabet.lower().split(',')

In [10]:
proxy = "223.135.156.183:8080"
edge_option = webdriver.EdgeOptions()
edge_option.add_argument(f"--proxy-server={proxy}")

driver = webdriver.Edge(options=edge_option)
try:
    driver.get("https://vi.glosbe.com/id/vi")
except WebDriverException as e:
    print(f'Error : {e.msg}')

In [7]:
driver.close()

In [5]:
def log_when_stop(file, word):
    with open(file, 'w') as f:
        word_index = word_dict[word[0].lower()].index(word)
        letter_index = alphabet.index(word[0].lower())
        f.write(f'{letter_index} {word_index-1}')

def search_word(driver:webdriver, word, delay = 3):
    MAXTRY = 1
    count = 0
    while True:
        try:
            try:
                myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.NAME,"q")))
            except TimeoutException:
                print( "Searching took too much time!")
            # driver.get("https://vi.glosbe.com/id/vi/" + word)
            search_bar = driver.find_element(By.NAME,"q")
            search_bar.clear()
            search_bar.send_keys(word)
            search_bar.send_keys(Keys.RETURN)
            break
        except NoSuchElementException:
            driver.refresh()
            count += 1
            if count >= MAXTRY:
                print("Encounter a capcha")
                log_when_stop("stop_log.txt", word)
                raise Exception
    
def get_translation(driver, div1, div2):
    lang1 = driver.find_elements(By.CSS_SELECTOR, div1)
    lang2 = driver.find_elements(By.CSS_SELECTOR, div2)

    return lang1,lang2

def write_to_csv(lang1, lang2, file):
    if len(lang1) < 1 or len(lang2) < 1:
        return
    with open(file, 'a', encoding='utf-8') as f:    
        for i, _ in enumerate(lang1):
            f.write(f'{lang1[i].text}ッ{lang2[i].text}\n')
            
def load_all(driver,button_css,try_again ,delay=3):
    while True:
        try:
            try:
                myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, button_css)))
            except TimeoutException:
                try:
                    print("try again")
                    load_more = driver.find_element(By.CSS_SELECTOR, try_again)
                    webdriver.ActionChains(driver).move_to_element(load_more).click(load_more).perform()
                except NoSuchElementException:
                    break
            load_more = driver.find_element(By.CSS_SELECTOR, button_css)
        
            webdriver.ActionChains(driver).move_to_element(load_more).click(load_more).perform()
                
            # time.sleep(2)
        except NoSuchElementException:
            print("no more")
            break

def continue_from_last(file):
    text:str
    with open(file,'r') as f:
        text = f.readline()
  
    return tuple(map(int,text.split()))



In [11]:
button_class = "button[class='button-xs']"
try_again_button_class = "button[class='m-auto text-button-xs']"
div1 = "div[class='w-1/2 dir-aware-pr-1 '][lang='id']"
div2 = "div[class='w-1/2 dir-aware-pl-1 '][lang='vi']"
log_file = "stop_log.txt"
link = "https://vi.glosbe.com/id/vi/"
csv_file = "result1.csv"

i,j = continue_from_last(log_file)
print(f'{i} {j}')
for letter in alphabet[i:]:
    if letter != alphabet[i]:
        j = 0
    for word in word_dict[letter][j:]:
        search_word(driver, word, delay=5)
        # driver.get(link + word)
        load_all(driver, button_class,try_again_button_class, delay=3)
        id,vi = get_translation(driver, div1, div2)
        write_to_csv(id, vi, csv_file)

1 2095


KeyboardInterrupt: 

In [ ]:
test = False
if test:
    search_word(driver,"cinta")
    delay = 3 # seconds
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button[class='button-xs']")))
        print( "Page is ready!")
    except TimeoutException:
        print( "Loading took too much time!")

In [7]:
word_dict['b'].index("brosur")

1800

In [ ]:
driver.refresh()